In [20]:
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.datasets import VOCDetection
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader


In [21]:
class VOCDataset(torch.utils.data.Dataset):
    def __init__(self, root, year="2007", image_set="train", transforms=None):
        self.dataset = VOCDetection(root, year=year, image_set=image_set, download=False)
        self.transforms = transforms


    def __getitem__(self, idx):
        img, target = self.dataset[idx]
        img = F.to_tensor(img)


        boxes = []
        labels = []
        for obj in target['annotation']['object']:
            bbox = obj['bndbox']
            boxes.append([
            float(bbox['xmin']),
            float(bbox['ymin']),
            float(bbox['xmax']),
            float(bbox['ymax'])
        ])
            labels.append(1) # single-class example


        target = {
        "boxes": torch.tensor(boxes, dtype=torch.float32),
        "labels": torch.tensor(labels, dtype=torch.int64)
        }


        return img, target


    def __len__(self):
        return len(self.dataset)

In [22]:
def collate_fn(batch):
    return tuple(zip(*batch))


train_dataset = VOCDataset("./data", image_set="train")
val_dataset = VOCDataset("./data", image_set="val")


train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 21  # 20 VOC classes + background

# Φτιάχνουμε το pretrained μοντέλο
model = fasterrcnn_resnet50_fpn(pretrained=True)

# Αντικαθιστούμε το box predictor
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(
    in_features, num_classes
)

# Στέλνουμε όλο το μοντέλο στο σωστό device
model = model.to(device)

In [24]:
optimizer = torch.optim.SGD(
    model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005
)


num_epochs = 10


for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0


    for images, targets in train_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()


        epoch_loss += losses.item()


    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.2f}")
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.2f}")

KeyboardInterrupt: 